In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iemocap-trainingdata/data_test.pt
/kaggle/input/iemocap-trainingdata/label_test.pt
/kaggle/input/iemocap-trainingdata/label_train.pt
/kaggle/input/iemocap-trainingdata/data_train.pt


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.optim as optim


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F

# Đường dẫn đến các file
data_train_path = "/kaggle/input/iemocap-trainingdata/data_train.pt"
label_train_path = "/kaggle/input/iemocap-trainingdata/label_train.pt"
data_test_path = "/kaggle/input/iemocap-trainingdata/data_test.pt"
label_test_path = "/kaggle/input/iemocap-trainingdata/label_test.pt"

# Load dữ liệu từ các file
data_train = torch.load(data_train_path)
label_train = torch.load(label_train_path)
data_test = torch.load(data_test_path)
label_test = torch.load(label_test_path)

# Chuẩn bị DataLoader cho dữ liệu huấn luyện và kiểm tra
batch_size = 32  # Thay đổi batch size theo nhu cầu của bạn

# Chuẩn bị DataLoader cho dữ liệu huấn luyện và kiểm tra
train_dataset = TensorDataset(data_train, label_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(data_test, label_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'data': self.data[idx], 'label': self.labels[idx]}
        return sample

# Create DataLoader for training and testing using the custom dataset
train_dataset_custom = CustomDataset(data_train, label_train)
train_loader_custom = DataLoader(train_dataset_custom, batch_size=batch_size, shuffle=True)

test_dataset_custom = CustomDataset(data_test, label_test)
test_loader_custom = DataLoader(test_dataset_custom, batch_size=batch_size, shuffle=False)

# Rest of your model and training code goes here
# Ensure you use the DataLoader to load data and labels during training and evaluation


In [18]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class ComplexNN(nn.Module):
#     def __init__(self, cnn_input_channels, cnn_output_channels, cnn_kernel_size, cnn_stride, 
#                  cnn_pool_kernel_size, cnn_pool_stride, hidden_size, output_size):
#         super(ComplexNN, self).__init__()
        
#         self.cnn = nn.Conv1d(cnn_input_channels, cnn_output_channels, cnn_kernel_size, stride=cnn_stride)
#         self.pool = nn.MaxPool1d(cnn_pool_kernel_size, stride=cnn_pool_stride)
        
#         self.input_size = cnn_output_channels * ((1500 - cnn_kernel_size) // cnn_pool_kernel_size + 1)
#         self.hidden = nn.Linear(self.input_size, hidden_size)
#         self.relu = nn.ReLU()
        
#         self.output = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = self.cnn(x)
#         x = self.pool(x)
#         x = x.view(x.size(0), -1)
#         x = self.hidden(x)
#         x = self.relu(x)
#         x = self.output(x)
#         x = F.softmax(x, dim=1)
#         return x

# # Define model parameters
# cnn_input_channels = 40
# cnn_output_channels = 16
# cnn_kernel_size = 5
# cnn_stride = 1
# cnn_pool_kernel_size = 2
# cnn_pool_stride = 2
# hidden_size = 128
# output_size = 3

# # Create an instance of the ComplexNN model
# model = ComplexNN(cnn_input_channels, cnn_output_channels, cnn_kernel_size, cnn_stride,
#                    cnn_pool_kernel_size, cnn_pool_stride, hidden_size, output_size)

# # Print the model architecture
# print(model)


ComplexNN(
  (cnn): Conv1d(40, 16, kernel_size=(5,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (hidden): Linear(in_features=11968, out_features=128, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=128, out_features=3, bias=True)
)


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ComplexNN(nn.Module):
    def __init__(self, cnn_input_channels, cnn_output_channels, cnn_kernel_size, cnn_stride, 
                 cnn_pool_kernel_size, cnn_pool_stride, hidden_size, output_size):
        super(ComplexNN, self).__init__()
        
        self.cnn = nn.Conv1d(cnn_input_channels, cnn_output_channels, cnn_kernel_size, stride=cnn_stride)
        self.pool = nn.MaxPool1d(cnn_pool_kernel_size, stride=cnn_pool_stride)
        
        # Calculate the input size for the linear layer based on the new input dimensions
        # Adjust for your specific input size (40, 1500)
        self.input_size = cnn_output_channels * ((1500 - cnn_kernel_size) // cnn_stride + 1) // cnn_pool_kernel_size
        self.hidden = nn.Linear(self.input_size, hidden_size)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.cnn(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        x = F.softmax(x, dim=1)
        return x

# Define updated model parameters for the new input dimensions (40, 1500)
cnn_input_channels = 40
cnn_output_channels = 16
cnn_kernel_size = 5
cnn_stride = 1
cnn_pool_kernel_size = 2
cnn_pool_stride = 2
hidden_size = 128
output_size = 3

# Create an instance of the updated ComplexNN model
updated_model = ComplexNN(cnn_input_channels, cnn_output_channels, cnn_kernel_size, cnn_stride,
                          cnn_pool_kernel_size, cnn_pool_stride, hidden_size, output_size)

# Print the updated model architecture
print(updated_model)


ComplexNN(
  (cnn): Conv1d(40, 16, kernel_size=(5,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (hidden): Linear(in_features=11968, out_features=128, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=128, out_features=3, bias=True)
)


In [30]:
import torch.optim as optim

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move the model to the appropriate device
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adjust the learning rate as needed

# Training loop
num_epochs = 100  # Adjust the number of epochs as needed

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for batch_data, batch_labels in train_loader:
        # Move the data to the appropriate device
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_data)
        loss = criterion(outputs, torch.argmax(batch_labels, dim=1))
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update running loss
        running_loss += loss.item()
    
    # Print the average loss for the epoch
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

print('Finished Training')


Using device: cuda
Epoch 1, Loss: 1.079695087892038
Epoch 2, Loss: 1.0785140578078214
Epoch 3, Loss: 1.0791045752151933
Epoch 4, Loss: 1.079695087261301
Epoch 5, Loss: 1.0779235482846619
Epoch 6, Loss: 1.0791045739537193
Epoch 7, Loss: 1.0796950897842488
Epoch 8, Loss: 1.0791045755305617
Epoch 9, Loss: 1.0791045739537193
Epoch 10, Loss: 1.077333032769501
Epoch 11, Loss: 1.0796950904149858
Epoch 12, Loss: 1.0779235467078194
Epoch 13, Loss: 1.0785140612768749
Epoch 14, Loss: 1.0791045755305617
Epoch 15, Loss: 1.0791045739537193
Epoch 16, Loss: 1.0791045752151933
Epoch 17, Loss: 1.0802856027764618
Epoch 18, Loss: 1.0791045752151933
Epoch 19, Loss: 1.078514061907612
Epoch 20, Loss: 1.0802856037225672
Epoch 21, Loss: 1.0791045739537193
Epoch 22, Loss: 1.0791045745844563
Epoch 23, Loss: 1.0785140606461379
Epoch 24, Loss: 1.0802856024610934
Epoch 25, Loss: 1.0796950888381434
Epoch 26, Loss: 1.079104574269088
Epoch 27, Loss: 1.077923545130977
Epoch 28, Loss: 1.07792354450024
Epoch 29, Loss: 1.

In [29]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.metrics import f1_score, accuracy_score

# # Check for GPU availability
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)

# # Move the model to the appropriate device
# model.to(device)

# # Loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust the learning rate as needed

# # Training loop
# num_epochs = 100  # Adjust the number of epochs as needed

# for epoch in range(num_epochs):
#     model.train()  # Set the model to training mode
#     running_loss = 0.0
#     total_predictions = []
#     true_labels = []
    
#     for batch_data, batch_labels in train_loader:
#         # Move the data to the appropriate device
#         batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
        
#         # Zero the parameter gradients
#         optimizer.zero_grad()
        
#         # Forward pass
#         outputs = model(batch_data)
#         loss = criterion(outputs, torch.argmax(batch_labels, dim=1))
        
#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()
        
#         # Update running loss
#         running_loss += loss.item()
        
#          # Update true labels and predictions for F1-score and accuracy
#         total_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
#         true_labels.extend(torch.argmax(batch_labels, dim=1).cpu().numpy())
    
#     # Calculate F1-score and accuracy
#     f1 = f1_score(true_labels, total_predictions, average='weighted')
#     acc = accuracy_score(true_labels, total_predictions)
    
#     # Print the average loss, F1-score, and accuracy for the epoch
#     print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, F1-score: {f1}, Accuracy: {acc}")

# print('Finished Training')

Using device: cuda
Epoch 1, Loss: 1.0796950900996174, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 2, Loss: 1.0796950875766693, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 3, Loss: 1.077923545130977, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 4, Loss: 1.0779235476539248, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 5, Loss: 1.079104574899825, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 6, Loss: 1.0785140600154008, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 7, Loss: 1.0779235479692932, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 8, Loss: 1.0785140600154008, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 9, Loss: 1.0791045752151933, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 10, Loss: 1.0791045764766674, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 11, Loss: 1.0791045739537193, F1-score: 0.3032

In [7]:
# # Set the model to evaluation mode
# model.eval()

# # Lists to store predictions and true labels for evaluation
# eval_predictions = []
# eval_true_labels = []

# # Iterate over the test dataset
# for eval_batch_data, eval_batch_labels in test_loader:
#     # Move the data to the appropriate device
#     eval_batch_data, eval_batch_labels = eval_batch_data.to(device), eval_batch_labels.to(device)
    
#     # Perform inference
#     with torch.no_grad():
#         eval_outputs = model(eval_batch_data)
    
#     # Append the predictions and true labels
#     eval_predictions.extend(torch.argmax(eval_outputs, dim=1).cpu().numpy())
#     eval_true_labels.extend(torch.argmax(eval_batch_labels, dim=1).cpu().numpy())

# # Calculate F1-score and accuracy on the test set
# eval_f1 = f1_score(eval_true_labels, eval_predictions, average='weighted')
# eval_acc = accuracy_score(eval_true_labels, eval_predictions)

# # Print the evaluation metrics
# print("Evaluation F1-score:", eval_f1)
# print("Evaluation Accuracy:", eval_acc)


Evaluation F1-score: 0.408752857653176
Evaluation Accuracy: 0.4893758300132802


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, accuracy_score

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move the model to the appropriate device
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust the learning rate as needed

# Initialize variables for early stopping and best model
early_stopping_counter = 0
best_test_loss = float('inf')
best_model_weights = None
loss_train = []
loss_test = []
f1_train = []
f1_test = []

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    total_predictions = []
    true_labels = []
    
    for batch_data, batch_labels in train_loader:
        # Move the data to the appropriate device
        batch_data, batch_labels = batch_data.to(device), batch_labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(batch_data)
        loss = criterion(outputs, torch.argmax(batch_labels, dim=1))
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Update running loss
        running_loss += loss.item()
        
        # Update true labels and predictions for F1-score and accuracy
        total_predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
        true_labels.extend(torch.argmax(batch_labels, dim=1).cpu().numpy())
    
    # Calculate F1-score and accuracy
    f1 = f1_score(true_labels, total_predictions, average='weighted')
    acc = accuracy_score(true_labels, total_predictions)
    
    # Print the average loss, F1-score, and accuracy for the epoch
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, F1-score: {f1}, Accuracy: {acc}")

    # Check if the test loss has improved for early stopping
    if running_loss < best_test_loss:
        best_test_loss = running_loss
        early_stopping_counter = 0
        best_model_weights = model.state_dict()
    else:
        early_stopping_counter += 1

    # Check for early stopping
    if early_stopping_counter >= 5:
        print("Early stopping triggered. No improvement for 5 consecutive epochs.")
        break

# Load the best model weights before saving
if best_model_weights is not None:
    model.load_state_dict(best_model_weights)

# Save the weights of the best model
torch.save(model.state_dict(), 'best_model_weights.pth')
print('Best model weights saved as best_model_weights.pth')

# Save the final model weights
torch.save(model.state_dict(), 'final_model_weights.pth')
print('Final model weights saved as final_model_weights.pth')

Using device: cuda
Epoch 1, Loss: 1.0796950882074063, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 2, Loss: 1.0785140631690857, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 3, Loss: 1.077923545130977, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 4, Loss: 1.0785140603307695, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 5, Loss: 1.0791045739537193, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 6, Loss: 1.0791045752151933, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 7, Loss: 1.0773330330848694, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 8, Loss: 1.0791045739537193, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 9, Loss: 1.0796950897842488, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 10, Loss: 1.0791045761612987, F1-score: 0.303258001841215, Accuracy: 0.4725219990038187
Epoch 11, Loss: 1.0791045764766674, F1-score: 0.303